# Walkthrough using napari
From raw image to plot the area through time of the wound

In [13]:
import napari
from magicgui import magicgui
from napari.types import ImageData, LabelsData
from enum import Enum

import numpy as np

In [2]:
viewer = napari.Viewer()

2022-06-26 11:27:29.474086: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-26 11:27:29.477287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 11:27:29.477312: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/clement/Documents/centuri-hackaton/venv/lib/python3.8/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your

In [3]:
from base_segmentation import Otsu, YAPIC, ChanVese
from utils import load_params, load_tiff, save_tiff
from pathlib import Path

In [4]:
# load all Segmentation Parameters

param_path_otsu = Path('../parameters/default_otsu.yml')
params_otsu = load_params(param_path_otsu)
segmentator_params_otsu = params_otsu["otsu"]
param_path_yapic = Path('../parameters/default_YAPIC.yml')
params_yapic = load_params(param_path_yapic)
segmentator_params_yapic = params_yapic["yapic"]
param_path_chanvese = Path('../parameters/default_chanvese.yml')
params_chanvese = load_params(param_path_chanvese)
segmentator_params_chanvese = params_chanvese["chanvese"]

In [5]:

import pathlib

@magicgui(
    call_button="Load tif image"
)
def image_loader(
    filename=pathlib.Path(params_otsu["input"]["path"])
) -> ImageData:
    im_axes = params_otsu["input"]["axes"]
    im = load_tiff(filename, im_axes)
    return im
# add our new magicgui widget to the viewer

viewer.window.add_dock_widget(image_loader)

In [15]:
segmentator_params_chanvese

{'disk_size': 4, 'iteration_nb': 10}

In [16]:
@magicgui(call_button="Segment input image using Chan Vese",)
def chanvese_widget(
    input_image: ImageData,
    disk_size:int=segmentator_params_chanvese["disk_size"],
    iteration_nb:int=segmentator_params_chanvese["iteration_nb"],
    nb_of_timestep_to_segment:int=10,
) -> LabelsData:
    """Compute Chan Vese segmentation"""
    mask = np.zeros(input_image.shape, dtype=bool)
    t = int(viewer.dims.point[0])
    input_image = input_image[t:t+nb_of_timestep_to_segment]

    segmentator_params_chanvese["disk_size"] = disk_size
    segmentator_params_chanvese["iteration_nb"] = iteration_nb
    segmentator = ChanVese(input_image, params=segmentator_params_chanvese)
    mask_t = segmentator.get_segmentation()
    mask[t:t+nb_of_timestep_to_segment] = mask_t
    return mask

# add our new magicgui widget to the viewer
viewer.window.add_dock_widget(chanvese_widget)


In [20]:
@magicgui(
    call_button="save tif image"
)
def save_tiff_wgt(
    mask_label: LabelsData,
    filename=pathlib.Path(params_otsu["output"]["path"]),
):
    im_axes = params_otsu["output"]["axes"]
    save_tiff(filename, mask_label, im_axes)
# add our new magicgui widget to the viewer

viewer.window.add_dock_widget(save_tiff_wgt)

In [8]:
@magicgui(call_button="Segment input image using Chan Vese",)
def find_holes_mask(
    input_image: ImageData,
    disk_size:int=segmentator_params_chanvese["disk_size"],
    iteration_nb:int=segmentator_params_chanvese["iteration_nb"],
    nb_of_timestep_to_segment:int=10,
) -> LabelsData:
    """Compute Chan Vese segmentation"""
    mask = np.zeros(input_image.shape, dtype=bool)
    t = int(viewer.dims.point[0])
    input_image = input_image[t:t+nb_of_timestep_to_segment]

    segmentator_params_chanvese["disk_size"] = disk_size
    segmentator_params_chanvese["iteration_nb"] = iteration_nb
    segmentator = ChanVese(input_image, params=segmentator_params_chanvese)
    mask_t = segmentator.get_segmentation()
    mask[t:t+nb_of_timestep_to_segment] = mask_t
    return mask

# add our new magicgui widget to the viewer
viewer.window.add_dock_widget(chanvese_widget)


(519.0, 288.0, 288.0)

In [ ]:
# load image

im_path = Path(params["input"]["path"])
im_axes = params["input"]["axes"]
im = load_tiff(im_path, im_axes)[:10] # take only a slice of the image for demo

In [14]:
viewer = napari.Viewer()

/home/clement/Documents/centuri-hackaton/venv/lib/python3.8/site-packages/napari_tools_menu/__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [33]:
from enum import Enum

import numpy
import napari
from napari.types import ImageData

from magicgui import magicgui

class Operation(Enum):
    add = yapic_wrapper
    subtract = numpy.subtract
    multiply = numpy.multiply
    divide = numpy.divide

test = [yapic_wrapper, find_outer_edge]
mode={"choices": ["reflect", "constant", "nearest", "mirror", "wrap"]},

# here's the magicgui!  We also use the additional
# `call_button` option

@magicgui(call_button="execute",
segmentation={"choices": ["yapic", "find_outer_edge"]})
def image_arithmetic(
    layerA: ImageData, segmentation: test, layerB: ImageData
) -> ImageData:
    """Add, subtracts, multiplies, or divides to image layers."""
    return segmentation

# create a viewer and add a couple image layers
viewer = napari.Viewer()
viewer.add_image(numpy.random.rand(20, 20), name="Layer 1")
viewer.add_image(numpy.random.rand(20, 20), name="Layer 2")

# add our new magicgui widget to the viewer

viewer.window.add_dock_widget(image_arithmetic)


# keep the dropdown menus in the gui in sync with the layer model
viewer.layers.events.inserted.connect(image_arithmetic.reset_choices)
viewer.layers.events.removed.connect(image_arithmetic.reset_choices)

napari.run()

TypeError: Annotated[t, ...]: t must be a type. Got [<function yapic_wrapper at 0x7fc2dc4c4160>, <function find_outer_edge at 0x7fc3345fd9d0>].

In [15]:
class Segmentation(Enum):
    """
    """
    yapic = yapic_wrapper
    find_outer_edge = find_outer_edge

In [27]:
@magicgui(call_button="execute")
def image_arithmetic(
    layerA: ImageData, segmentation: Segmentation="yapic"
) -> ImageData:
    """Add, subtracts, multiplies, or divides to image layers."""
    return segmentation.value(layerA, layerB)



viewer.layers.events.inserted.connect(image_arithmetic.reset_choices)
viewer.layers.events.removed.connect(image_arithmetic.reset_choices)

ValueError: 'yapic' is not a valid choice. must be in ()

In [19]:
import math
from enum import Enum
from magicgui import magicgui

# dropdown boxes are best made by creating an enum
class Medium(Enum):
    Glass = 1.520
    Oil = 1.515
    Water = 1.333
    Air = 1.0003

# decorate your function with the ``@magicgui`` decorator
@magicgui(call_button="calculate")
def snells_law(aoi=30.0, n1=Medium.Glass, n2=Medium.Water, degrees=True):
    aoi = math.radians(aoi) if degrees else aoi
    try:
        result = math.asin(n1.value * math.sin(aoi) / n2.value)
        return math.degrees(result) if degrees else result
    except ValueError:
        # beyond the critical angle
        return "Total internal reflection!"


In [17]:
viewer.add_image(np.random.rand(20, 20), name="Layer 1")
viewer.add_image(np.random.rand(20, 20), name="Layer 2")

<Image layer 'Layer 2' at 0x7fc26db4dfd0>

In [25]:
viewer.window.add_dock_widget(image_arithmetic)